In [1]:
import pandas as pd 
import numpy as np
import os
import csv
from pathlib import Path
import datetime as dt

In [2]:
filename='ACL_2021-07-20.csv'

In [3]:
def avg_time(datetimes):
    total = sum(dt.hour * 3600 + dt.minute * 60 + dt.second for dt in datetimes)
    avg = total / len(datetimes)
    minutes, seconds = divmod(int(avg), 60)
    hours, minutes = divmod(minutes, 60)
    return avg

In [321]:
def attendance(filename):
    date = filename.split('_')[1]
    date = date.split('.')[0]
    subject = filename.split('_')[0]
    data = pd.read_csv(subject + '.csv')
    df = pd.read_csv(filename)
    df['Time in Call'] = pd.to_datetime(df['Time in Call'])
    df['Time in Call']=df['Time in Call'].dt.time
    times = df['Time in Call'].to_list()
    df['totalTime']=df['Time in Call'].apply(lambda x:(x.hour*3600 + x.minute*60 +x.second))
    avg = avg_time(times)
    threshold = 0.5*avg
    names=df['Full Name'].to_list()
    data[date]=data['Full Name'].apply(lambda x: 1 if x in names else "absent")  
    data[date]=df['totalTime'].apply(lambda x: 1 if x >threshold else "absent")
    data.to_csv(subject + '.csv',index=False)

In [327]:
attendance(filename)

In [111]:
date = filename.split('_')[1]
date = date.split('.')[0]
subject = filename.split('_')[0]
data = pd.read_csv(subject + '.csv')
df = pd.read_csv(filename)

names=df['Full Name'].to_list()
data[date]=data['Full Name'].apply(lambda x: 1 if x in names else "absent")
df['Time in Call'] = pd.to_datetime(df['Time in Call'])
df['Time in Call']=df['Time in Call'].dt.time
# times = df['Time in Call'].to_list()
# df['totalTime']=df['Time in Call'].apply(lambda x:(x.hour*3600 + x.minute*60 +x.second))
# avg = avg_time(times)
# threshold = 0.5*avg
# data[date]=df['totalTime'].apply(lambda x: 1 if x >threshold else "absent")

KeyError: 'totalTime'

In [112]:
date = filename.split('_')[1]
date = date.split('.')[0]
subject = filename.split('_')[0]
data = pd.read_csv(subject + '.csv')
df = pd.read_csv(filename)

names=df['Full Name'].to_list()
df['Time in Call'] = pd.to_datetime(df['Time in Call'])
df['Time in Call']=df['Time in Call'].dt.time
data[date]=data['Full Name'].apply(lambda x: 1 if x in names else "")
fullnames=data['Full Name'].to_list()
# df['totalTime']=df['Time in Call'].apply(lambda x:(x.hour*3600 + x.minute*60 +x.second))
# data['totalTime']=data[data['Full Name']]
# avg = avg_time(times)
# threshold = 0.5*avg
# data.tail(20)
# data.to_csv(subject + '.csv',index=False)
data.tail(20)

,Full Name,2021-07-20
37,SUBHASELVI A,1
38,SUDARSAN M,1
39,SUNIL KUMAR YALAMURI,1
40,SUPRIYA VARSHINI,1
41,SYED OWAIS,
42,SYED ROSHAN 19EC1100,1
43,SYSCON JOSE A J,1
44,Sandanalakshmi sridaran,1
45,Srinivasamurthy C,1
46,TAMIRI SATYA,1


time = [sum(time.hour * 3600 + time.minute * 60 + time.second) for time in times]

In [125]:
list1=[]
for name in data["Full Name"]:
    a=df[df['Full Name']==name]['Time in Call']
    list1.append(a)


In [73]:
finallist=list(map(lambda x:0 if x.empty==True else x,list1)

SyntaxError: unexpected EOF while parsing (<ipython-input-73-7a62e1713f0e>, line 1)

In [127]:
for x in iter(list1):
    if x.empty ==True:
        x=pd.Series(0)

In [128]:
list1

[0    00:49:37
 Name: Time in Call, dtype: object,
 1    00:55:20
 Name: Time in Call, dtype: object,
 2    00:50:42
 Name: Time in Call, dtype: object,
 3    01:04:21
 Name: Time in Call, dtype: object,
 4    00:50:40
 Name: Time in Call, dtype: object,
 5    00:51:06
 Name: Time in Call, dtype: object,
 6    00:58:43
 Name: Time in Call, dtype: object,
 7    01:00:48
 Name: Time in Call, dtype: object,
 8    00:51:15
 Name: Time in Call, dtype: object,
 9    01:04:46
 Name: Time in Call, dtype: object,
 10    00:55:52
 Name: Time in Call, dtype: object,
 11    00:58:58
 Name: Time in Call, dtype: object,
 12    00:57:35
 Name: Time in Call, dtype: object,
 13    00:55:56
 Name: Time in Call, dtype: object,
 14    01:02:50
 Name: Time in Call, dtype: object,
 15    01:04:25
 Name: Time in Call, dtype: object,
 16    00:03:03
 Name: Time in Call, dtype: object,
 17    01:01:23
 Name: Time in Call, dtype: object,
 18    01:02:18
 Name: Time in Call, dtype: object,
 19    00:54:38
 Name:

In [82]:
list1[48].empty==True


True

GET ACL.csv

In [20]:
df1=pd.read_csv(filename)
df1.drop(["First Seen","Time in Call"],axis=1,inplace=True)
df1.to_csv("ACL.csv",index=False)

In [302]:
date = filename.split('_')[1]
date = date.split('.')[0]
df1=pd.read_csv("ACL.csv")
df1.drop(["totalTime",date],axis=1,inplace=True)
df1.to_csv("ACL.csv",index=False)